In [ ]:
import numpy as np

In [ ]:
d = {
    "key": "value",
    "key1": "value1",
}

In [ ]:
from box import ConfigBox

In [ ]:
d2 = ConfigBox(d)

In [ ]:
d2.key1

In [ ]:
def get_product(x:int, y:int)->int:
    return x*y

In [ ]:
get_product(x = 2, y = 4)

In [ ]:
get_product(x = 2, y = "4")

In [ ]:
from ensure import ensure_annotations

In [ ]:
@ensure_annotations
def get_product(x:int, y:int)->int:
    return x*y

In [ ]:
get_product(x = 2, y = "4")

In [ ]:
get_product(x = 2, y = 4)

In [1]:
pwd

'c:\\Prajesh\\personal\\ML_learning\\my_work\\kubeflow_app\\kubeflow_ML_app\\ML_flow_app\\research'

In [2]:
cd ..

c:\Prajesh\personal\ML_learning\my_work\kubeflow_app\kubeflow_ML_app\ML_flow_app


In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from mlFlowProject.constants import *
from mlFlowProject.utils.common import read_yaml, create_directories
import os

In [30]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        # filedir, filename = os.path.split(CONFIG_FILE_PATH)
        # config_filepath = Path(filedir + '/' + filename)
        print(config_filepath)
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [31]:
# dummy = ConfigurationManager()

In [32]:
import os
import urllib.request as request
import zipfile
from mlFlowProject import logger
from mlFlowProject.utils.common import get_size

In [45]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [48]:
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_data()
    # data_ingestion.extract_zip_file()
except Exception as e:
    raise e

config\config.yaml
[2023-08-05 23:34:21,504: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-05 23:34:21,506: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-05 23:34:21,507: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-05 23:34:21,508: INFO: common: created directory at: artifacts]
[2023-08-05 23:34:21,509: INFO: common: created directory at: artifacts/data_ingestion]
[2023-08-05 23:34:22,030: INFO: 2174002776: artifacts/data_ingestion/data.csv download! with following info: 
content-disposition: attachment; filename=SAIL.NS.csv
content-type: text/csv;charset=utf-8
vary: Origin
cache-control: private, max-age=10, stale-while-revalidate=20
y-rid: 5sdcj9hict3p6
x-yahoo-request-id: 5sdcj9hict3p6
x-request-id: cdbb6cf4-ba47-4cba-a3c1-0054f6e08082
x-envoy-upstream-service-time: 124
date: Sat, 05 Aug 2023 18:04:22 GMT
server: ATS
x-envoy-decorator-operation: finance-chart-api--mtls-production-sg3.finance-k8s.svc.yahoo.local: